# Evaluation using RAGAS

RAGAS (Retrieval-Augmented Generation Assessment Suite) is an open-source framework designed to systematically evaluate retrieval-augmented generation (RAG) pipelines. It measures key aspects of performance including faithfulness (factual grounding), answer relevancy, context precision, context recall, and overall answer correctness. By combining LLM-based evaluations with traditional retrieval metrics, RAGAS provides a structured and quantitative way to assess how well a RAG system retrieves relevant information and generates accurate, context-based answers.

https://docs.ragas.io/en/stable/

In [36]:
from datasets import Dataset

from ragas.metrics import (
    faithfulness,
    answer_relevancy,
    context_precision,
    context_recall,
    answer_correctness,
)
from ragas import evaluate

# import dotenv, use try except as plan on deploying this project via hugginface spaces
try:
    from dotenv import load_dotenv
    load_dotenv()
except ImportError:
    pass  # dotenv not used in deployment

import os

import pandas as pd

In [37]:
# gets open ai key
openai_api = os.getenv("OPENAI_API_KEY")

# this tells RAGAS / LangChain / OpenAI Python SDK to use the key
# it uses gpt-3.5-turbo by default
os.environ["OPENAI_API_KEY"] = openai_api

### Evaluation using simple chunking

In [38]:
# each question is passed through the RAG model and the answer is what was generated
data = [
    {
        "question": "What is Reform UK's policy on immigration?",
        "answer": "Reform UK's policy on immigration includes freezing non-essential immigration, stopping illegal immigrants with a 4 Point Plan, securing detention for all illegal migrants, immediate deportation for foreign criminals, barring student dependents, stopping health tourism and immediate access to benefits, and implementing an employer immigration tax.",
        "contexts": [
            "Reform UK proposes to freeze non-essential immigration, stop the small boats, and deport illegal migrants. They also plan to impose a five-year residency requirement for claiming benefits."
        ],
        "ground_truth": "Freeze non-essential immigration, stop small boats, deport illegal migrants, five-year residency rule for benefits."
    },
    {
        "question": "How do the Liberal Democrats plan to improve access to GPs?",
        "answer": """The Liberal Democrats plan to improve access to GPs by giving everyone the right to see a GP within seven days, or within 24 hours if they urgently need to. They also aim to achieve this by increasing the number of full-time equivalent GPs by 8,000, half through boosting recruitment and half from retaining more experienced GPs. Additionally, they plan to free up GPs' time by giving more prescribing rights and public health advisory services to qualified pharmacists, nurse practitioners, and paramedics, as well as establishing a universal 
        24/7 GP booking system and removing top-down bureaucracy to allow practices to hire the staff they need and invest in training.""",
        "contexts": [
            "Liberal Democrats will give everyone a new right to see a GP within seven days, or 24 hours if urgent, with extra doctors to support this plan."
        ],
        "ground_truth": "Right to see a GP within seven days or 24 hours if urgent, supported by hiring more doctors."
    },
    {
        "question": "What are Labour's five national missions according to the 2024 manifesto?",
        "answer": """  1) Kickstart economic growth to secure the highest sustained growth in the G7
                        2) Make Britain a clean energy superpower
                        3) Take back our streets by halving serious violent crime
                        4) Break down barriers to opportunity
                        5) Build an NHS fit for the future""",
        "contexts": [
            "Labour’s five missions are: kickstart economic growth, make Britain a clean energy superpower, take back our streets, break down barriers to opportunity, and build an NHS fit for the future."
        ],
        "ground_truth": "Kickstart economic growth, clean energy superpower, take back streets, break barriers to opportunity, build a future-ready NHS."
    },
    {
        "question": "What is the Green Party's position on NHS funding?",
        "answer": """The Green Party's position on NHS funding is to increase taxes on the super-rich to fund the national health service properly, fix the broken care system that is causing problems for the NHS, and ensure a fully public health service with no profit motive involved. They estimate that additional annual expenditure of £8bn in the first full year of the next parliament, rising to a total of £28bn in 2030, is needed for the NHS in England. They also estimate that additional capital spending of at least £20bn is needed over the next five years. Additionally, the Green Party plans to invest £20bn in NHS budgets over the life of the parliament for hospital building and repair, and to reduce hospital waiting lists by giving Hospital Trusts long-term funding commitments.""",
        "contexts": [
            "The Green Party pledges to increase NHS spending by £8 billion in the first year, rising to £28 billion by 2030, and keep the NHS fully public."
        ],
        "ground_truth": "Increase NHS spending significantly and ensure the NHS remains fully public."
    },
    {
        "question": "How do the Conservatives plan to reduce immigration?",
        "answer": """The Conservatives plan to reduce immigration by introducing a binding, legal cap on migration, setting it on work and family visas to protect public services while bringing in 
                     the necessary skills for businesses and the NHS. They will set the cap at a level that considers the costs and benefits of migration, with the cap decreasing every year and requiring an annual vote in Parliament to control immigration numbers. They also aim to deter illegal migration by implementing measures such as removing the incentive to come illegally, running 
                     a process of permanently removing illegal migrants to Rwanda, cracking down on organised immigration crime, and reforming asylum rules.""",
        "contexts": [
            "The Conservatives plan to introduce a legal cap on migration numbers, remove illegal migrants to Rwanda, and rewrite asylum treaties."
        ],
        "ground_truth": "Legal migration cap, Rwanda deportation policy, rewrite asylum treaties."
    },
    {
        "question": "What is Reform UK's policy on NHS waiting lists?",
        "answer": "Reform UK's policy on NHS waiting lists includes critical reforms needed in the first 100 days such as ending doctor and nurse shortages, utilizing independent healthcare capacity, providing tax relief on private healthcare and insurance, and implementing a new NHS voucher scheme for patients who can't see a GP within a certain timeframe. They also aim to improve efficiency, cut waste, and unnecessary managers in the NHS to reduce waiting times.",
        "contexts": [
            "Reform UK pledges to achieve zero NHS waiting lists by reforming healthcare delivery, cutting back-office waste, and offering tax breaks to doctors and nurses to tackle staffing shortages."
        ],
        "ground_truth": "Zero NHS waiting lists through healthcare reform, cutting waste, and tax breaks for staff."
    },
    {
        "question": "How will the Liberal Democrats tackle climate change?",
        "answer": """The Liberal Democrats plan to tackle climate change by putting it at the heart of a new industrial strategy. They will cut emissions and bills with an emergency Home Energy Upgrade programme, drive a rooftop solar revolution, and invest in clean energy, transport, and industry. They also aim to restore nature, tackle toxic air pollution, and provide skills training, incentives, and advice to help families and businesses transition to net zero. Additionally, they plan to regulate financial services to encourage climate-friendly investments, support British industry to cut emissions, and provide more advice to companies on cutting emissions.""",
        "contexts": [
            "The Liberal Democrats will drive a rooftop solar revolution, invest in clean energy and transport, ban water companies from dumping sewage, and implement an emergency Home Energy Upgrade programme."
        ],
        "ground_truth": "Invest in clean energy, rooftop solar, ban sewage dumping, and upgrade home energy."
    },
    {
        "question": "What energy policy is proposed by Labour in their 2024 manifesto?",
        "answer": """Labour proposes to set up Great British Energy to cut bills for good, achieve energy independence from dictators like Putin, create 650,000 new high-quality jobs, warm homes to slash fuel poverty, and force water companies to clean up rivers. They also plan to deploy more distributed production capacity through their Local Power Plan, partnering with energy 
                     companies, local authorities, and co-operatives to install clean power projects such as onshore wind, solar, and hydropower projects. They aim to ensure local people benefit directly from this energy production and reform the energy system to put consumers first and attract the investment needed to cut bills.""",
        "contexts": [
            "Labour proposes creating a publicly-owned clean power company, Great British Energy, to cut bills, boost energy security, and achieve zero-carbon electricity by 2030."
        ],
        "ground_truth": "Create Great British Energy, deliver zero-carbon electricity by 2030."
    },
    {
        "question": "What is the Green Party’s approach to public transport?",
        "answer": """The Green Party's approach to public transport includes championing public transport for every community to thrive, increasing annual public subsidies for rail and bus travel, 
                     investing in new cycleways and footpaths, bringing the railways back into public ownership, giving local authorities control over and funding for improved bus services, and ensuring investment in new cycleways and footpaths. They also aim to increase public transport subsidies to make it reliable, frequent, accessible, and affordable, including providing free bus travel for under-18s. Additionally, they oppose all new road building plans and aim to shift the transport system away from cars and roads by promoting the use of electric vehicles and active travel options like walking and cycling.""",
        "contexts": [
            "The Green Party plans to invest in public transport, including rail, bus, and active travel infrastructure, to reduce car dependency and cut emissions."
        ],
        "ground_truth": "Invest in rail, bus, and active travel infrastructure to cut emissions and car dependency."
    },
    {
        "question": "How do the Conservatives propose to support first-time home buyers?",
        "answer": """The Conservatives propose to support first-time home buyers by ensuring the majority of first-time buyers pay no Stamp Duty at all, making permanent the increase to the threshold at which first-time buyers pay Stamp Duty to £425,000, launching a new and improved Help to Buy scheme to provide first-time buyers with an equity loan of up to 20% towards the cost of a new build home, and continuing the Mortgage Guarantee Scheme. They also plan to legislate for new 'Local Connection' and 'UK Connection' tests for social housing in England, implement a 'three strikes and you're out' expectation for social housing landlords for anti-social behavior, and protect family homes from higher tax.""",
        "contexts": [
            "The Conservatives propose abolishing Stamp Duty for homes up to £425,000 for first-time buyers and introducing a new Help to Buy scheme."
        ],
        "ground_truth": "Abolish Stamp Duty for homes up to £425,000 and introduce Help to Buy scheme."
    }
]

In [39]:
# creates hugging face dataset
ragas_dataset = Dataset.from_list(data)

In [40]:
# sets list of metrics to evaluate by
metrics = [
    faithfulness,
    answer_relevancy,
    context_precision,
    context_recall,
    answer_correctness
]

### Results

In [41]:
results = evaluate(ragas_dataset, metrics=metrics)
print(results)

Evaluating: 100%|██████████████████████████████████████████████████████████████████████| 50/50 [00:48<00:00,  1.04it/s]


{'faithfulness': 0.2081, 'answer_relevancy': 0.9536, 'context_precision': 1.0000, 'context_recall': 1.0000, 'answer_correctness': 0.4958}


In [42]:
# creates dataframe of all results for each question
scores_per_sample = pd.DataFrame(results.scores)

# Show per sample scores
scores_per_sample

,faithfulness,answer_relevancy,context_precision,context_recall,answer_correctness
0,0.142857,0.957125,1.0,1.0,0.600007
1,0.166667,0.991475,1.0,1.0,0.425542
2,0.600000,0.863745,1.0,1.0,0.985265
3,0.375000,1.000000,1.0,1.0,0.367492
4,0.363636,0.955531,1.0,1.0,0.317546
5,0.333333,0.948528,1.0,1.0,0.333163
6,0.100000,0.973649,1.0,1.0,0.508517
7,0.000000,0.904589,1.0,1.0,0.330308
8,0.000000,0.997762,1.0,1.0,0.530288
9,0.000000,0.943372,1.0,1.0,0.559429


In [43]:
# Calculate average across all samples
mean_scores = scores_per_sample.mean()
print("\nAverage Scores:")
print(mean_scores)


Average Scores:
faithfulness          0.208149
answer_relevancy      0.953578
context_precision     1.000000
context_recall        1.000000
answer_correctness    0.495756
dtype: float64


## Added new prompt to chat code and re-evaluating

In [44]:
data = [
    {
        "question": "What is Reform UK's policy on immigration?",
        "answer": """Reform UK's policy on immigration includes freezing non-essential immigration, stopping illegal immigrants with a 4 Point Plan, securing detention for all illegal migrants, immediate deportation for foreign criminals, barring student dependents, stopping health tourism and immediate access to benefits, and implementing an employer immigration tax. They aim to secure Britain's borders to protect wages, public services, and British culture and values.""",
        "contexts": [
            "Reform UK proposes to freeze non-essential immigration, stop the small boats, and deport illegal migrants. They also plan to impose a five-year residency requirement for claiming benefits."
        ],
        "ground_truth": "Freeze non-essential immigration, stop small boats, deport illegal migrants, five-year residency rule for benefits."
    },
    {
        "question": "How do the Liberal Democrats plan to improve access to GPs?",
        "answer": """The Liberal Democrats plan to improve access to GPs by giving everyone the right to see a GP within seven days, or within 24 hours if they urgently need to, by increasing the number of full-time equivalent GPs by 8,000, half by boosting recruitment and half from retaining more experienced GPs. They also aim to free up GPs' time by giving more prescribing rights and public health advisory services to qualified pharmacists, nurse practitioners, and paramedics, as well as establishing a universal 24/7 GP booking system and removing top-down 
bureaucracy to let practices hire the staff they need and invest in training.""",
        "contexts": [
            "Liberal Democrats will give everyone a new right to see a GP within seven days, or 24 hours if urgent, with extra doctors to support this plan."
        ],
        "ground_truth": "Right to see a GP within seven days or 24 hours if urgent, supported by hiring more doctors."
    },
    {
        "question": "What are Labour's five national missions according to the 2024 manifesto?",
        "answer": """1) Kickstart economic growth to secure the highest sustained growth in the G7.
2) Make Britain a clean energy superpower.
3) Take back our streets by halving serious violent crime.
4) Break down barriers to opportunity by reforming childcare and education systems.
5) Build an NHS fit for the future.""",
        "contexts": [
            "Labour’s five missions are: kickstart economic growth, make Britain a clean energy superpower, take back our streets, break down barriers to opportunity, and build an NHS fit for the future."
        ],
        "ground_truth": "Kickstart economic growth, clean energy superpower, take back streets, break barriers to opportunity, build a future-ready NHS."
    },
    {
        "question": "What is the Green Party's position on NHS funding?",
        "answer": """The Green Party's position on NHS funding includes increasing taxes on the super-rich to fund the national health service properly, fixing the broken care system that impacts the NHS, ensuring fair pay for NHS staff, restoring local council budgets for public health, and advocating for additional annual expenditure of £8bn in the first full year of the next 
parliament, rising to a total of £28bn in 2030. They also emphasize the need for additional capital spending of at least £20bn over the next five years to address the long-term underfunding of the NHS.""",
        "contexts": [
            "The Green Party pledges to increase NHS spending by £8 billion in the first year, rising to £28 billion by 2030, and keep the NHS fully public."
        ],
        "ground_truth": "Increase NHS spending significantly and ensure the NHS remains fully public."
    },
    {
        "question": "How do the Conservatives plan to reduce immigration?",
        "answer": """The Conservatives plan to reduce immigration by establishing a binding, legal cap on migration, set on work and family visas, with the cap falling every year of the next Parliament and cannot be breached. They will give parliament an annual vote on the level of the cap to ensure immigration numbers are controlled. They also aim to deter illegal migration by 
permanently removing illegal migrants to Rwanda, ending the legal merry-go-round of spurious challenges to block removal, cracking down on organised immigration crime, reforming asylum rules, and returning people with no right to be in the UK to their own country. Additionally, they plan to give parliament control over how many places are offered on safe and legal routes to support refugees in genuine need.""",
        "contexts": [
            "The Conservatives plan to introduce a legal cap on migration numbers, remove illegal migrants to Rwanda, and rewrite asylum treaties."
        ],
        "ground_truth": "Legal migration cap, Rwanda deportation policy, rewrite asylum treaties."
    },
    {
        "question": "What is Reform UK's policy on NHS waiting lists?",
        "answer": """Reform UK's policy on NHS waiting lists includes critical reforms needed in the first 100 days, such as ending doctor and nurse shortages, utilizing independent healthcare capacity, providing tax relief on private healthcare and insurance, and implementing a new NHS voucher scheme for patients who can't see a GP within a specified timeframe. They also aim to improve efficiency, cut waste, and unnecessary managers in the NHS to address waiting times.""",
        "contexts": [
            "Reform UK pledges to achieve zero NHS waiting lists by reforming healthcare delivery, cutting back-office waste, and offering tax breaks to doctors and nurses to tackle staffing shortages."
        ],
        "ground_truth": "Zero NHS waiting lists through healthcare reform, cutting waste, and tax breaks for staff."
    },
    {
        "question": "How will the Liberal Democrats tackle climate change?",
        "answer": """The Liberal Democrats will tackle climate change by putting it at the heart of a new industrial strategy, cutting emissions and bills with an emergency Home Energy Upgrade programme, driving a rooftop solar revolution, investing in clean energy, transport, and industry, restoring nature, tackling toxic air pollution, providing skills training, incentives, and advice to help families and businesses transition to net zero. They will also regulate financial services to encourage climate-friendly investments, support British industry to cut emissions, and provide more advice to companies on cutting emissions.""",
        "contexts": [
            "The Liberal Democrats will drive a rooftop solar revolution, invest in clean energy and transport, ban water companies from dumping sewage, and implement an emergency Home Energy Upgrade programme."
        ],
        "ground_truth": "Invest in clean energy, rooftop solar, ban sewage dumping, and upgrade home energy."
    },
    {
        "question": "What energy policy is proposed by Labour in their 2024 manifesto?",
        "answer": """Labour's energy policy proposal in their 2024 manifesto includes setting up Great British Energy to cut bills for good, achieving energy independence from dictators like Putin, creating 650,000 new high-quality jobs, ensuring warmer homes to slash fuel poverty, and forcing water companies to clean up rivers. They also plan to deploy more distributed production capacity through their Local Power Plan, partnering with energy companies, local authorities, and co-operatives to install clean power projects like onshore wind, solar, and hydropower projects. They aim to invite communities to come forward with projects and work with local leaders and devolved governments to ensure local people benefit directly from energy production. Additionally, Labour will ensure a tougher system of regulation that puts consumers first and attracts the investment needed to cut bills.""",
        "contexts": [
            "Labour proposes creating a publicly-owned clean power company, Great British Energy, to cut bills, boost energy security, and achieve zero-carbon electricity by 2030."
        ],
        "ground_truth": "Create Great British Energy, deliver zero-carbon electricity by 2030."
    },
    {
        "question": "What is the Green Party’s approach to public transport?",
        "answer": """The Green Party's approach to public transport includes championing public transport for every community to thrive, increasing annual public subsidies for rail and bus travel to £10bn by the end of the next parliament, investing an additional £19bn over five years to improve public transport, supporting electrification, investing in new cycleways and footpaths, bringing the railways back into public ownership, giving local authorities control over and funding for improved bus services, and ensuring £2.5bn a year is invested in new cycleways and footpaths. They also aim to empower local authorities to run bus services themselves if they see fit and provide a service that meets their community’s needs.""",
        "contexts": [
            "The Green Party plans to invest in public transport, including rail, bus, and active travel infrastructure, to reduce car dependency and cut emissions."
        ],
        "ground_truth": "Invest in rail, bus, and active travel infrastructure to cut emissions and car dependency."
    },
    {
        "question": "How do the Conservatives propose to support first-time home buyers?",
        "answer": """The Conservatives propose to support first-time home buyers by ensuring the majority of first-time buyers pay no Stamp Duty at all, making permanent the increase to the threshold at which first-time buyers pay Stamp Duty to £425,000 from £300,000, launching a new and improved Help to Buy scheme to provide first-time buyers with an equity loan of up to 20% towards the cost of a new build home, and continuing the Mortgage Guarantee Scheme to help more households buy a home.""",
        "contexts": [
            "The Conservatives propose abolishing Stamp Duty for homes up to £425,000 for first-time buyers and introducing a new Help to Buy scheme."
        ],
        "ground_truth": "Abolish Stamp Duty for homes up to £425,000 and introduce Help to Buy scheme."
    }
]


In [45]:
# creates hugging face dataset
ragas_dataset = Dataset.from_list(data)

In [46]:
# sets list of metrics to evaluate by
metrics = [
    faithfulness,
    answer_relevancy,
    context_precision,
    context_recall,
    answer_correctness
]

### Results

In [47]:
results = evaluate(ragas_dataset, metrics=metrics)
print(results)

Evaluating: 100%|██████████████████████████████████████████████████████████████████████| 50/50 [00:45<00:00,  1.09it/s]


{'faithfulness': 0.1851, 'answer_relevancy': 0.9656, 'context_precision': 1.0000, 'context_recall': 1.0000, 'answer_correctness': 0.6259}


In [48]:
# creates dataframe of all results for each question
scores_per_sample = pd.DataFrame(results.scores)

# Show per sample scores
scores_per_sample

,faithfulness,answer_relevancy,context_precision,context_recall,answer_correctness
0,0.181818,0.957125,1.0,1.0,0.505228
1,0.133333,0.991475,1.0,1.0,0.440712
2,0.400000,0.868261,1.0,1.0,0.985049
3,0.250000,1.000000,1.0,1.0,0.884306
4,0.250000,0.953771,1.0,1.0,0.401627
5,0.250000,0.948528,1.0,1.0,0.342693
6,0.100000,0.986722,1.0,1.0,0.444310
7,0.000000,0.966684,1.0,1.0,0.315604
8,0.000000,0.997762,1.0,1.0,0.964523
9,0.285714,0.985366,1.0,1.0,0.975265


In [49]:
# Calculate average across all samples
mean_scores = scores_per_sample.mean()
print("\nAverage Scores:")
print(mean_scores)


Average Scores:
faithfulness          0.185087
answer_relevancy      0.965569
context_precision     1.000000
context_recall        1.000000
answer_correctness    0.625932
dtype: float64


## Used more detailed chunking system to help with retrieval

In [50]:
data = [
    {
        "question": "What is Reform UK's policy on immigration?",
        "answer": """Reform UK's policy on immigration includes freezing non-essential immigration, implementing strict limits on immigration to relieve pressure on housing, public services, increase wages, and protect British culture, identity, and values. They also have a 4 Point Plan to stop boats and only allow essential skills, mainly in healthcare, as exceptions.""",
        "contexts": [
            "Reform UK proposes to freeze non-essential immigration, stop the small boats, and deport illegal migrants. They also plan to impose a five-year residency requirement for claiming benefits."
        ],
        "ground_truth": "Freeze non-essential immigration, stop small boats, deport illegal migrants, five-year residency rule for benefits."
    },
    {
        "question": "How do the Liberal Democrats plan to improve access to GPs?",
        "answer": """The Liberal Democrats plan to improve access to GPs by giving everyone the right to see a GP within seven days, or within 24 hours if they urgently need to, with the addition of 8,000 more GPs to deliver on this commitment. They also aim to introduce a universal 24/7 GP booking system, remove top-down bureaucracy to allow practices to hire the staff they need, and establish a Strategic Small Surgeries Fund to sustain services in rural and remote areas.""",
        "contexts": [
            "Liberal Democrats will give everyone a new right to see a GP within seven days, or 24 hours if urgent, with extra doctors to support this plan."
        ],
        "ground_truth": "Right to see a GP within seven days or 24 hours if urgent, supported by hiring more doctors."
    },
    {
        "question": "What are Labour's five national missions according to the 2024 manifesto?",
        "answer": """1) Kickstart economic growth to secure the highest sustained growth in the G7 – with good jobs and productivity growth in every part of the country making everyone, not just a few, better off.
2) Make Britain a clean energy superpower to cut bills, create jobs and deliver security with cheaper, zero-carbon electricity by 2030, accelerating to net zero.
3) Take back our streets by halving serious violent crime and raising confidence in the police and criminal justice system to its highest levels.
4) Break down barriers to opportunity by reforming our childcare and education systems, to make sure there is no class ceiling on the ambitions of young people in Britain.
5) Build an NHS fit for the future that is there when people need it; with fewer lives lost to the biggest killers; in a fairer Britain, where everyone lives well for longer.""",
        "contexts": [
            "Labour’s five missions are: kickstart economic growth, make Britain a clean energy superpower, take back our streets, break down barriers to opportunity, and build an NHS fit for the future."
        ],
        "ground_truth": "Kickstart economic growth, clean energy superpower, take back streets, break barriers to opportunity, build a future-ready NHS."
    },
    {
        "question": "What is the Green Party's position on NHS funding?",
        "answer": """The Green Party is committed to fully funding the NHS and keeping it public. They advocate for increasing NHS budgets to meet the demands of an ageing population, reducing waiting lists, guaranteeing rapid access to GPs and dentists, boosting the pay of NHS staff, and restoring local council budgets for public health. They also plan to invest £20bn in NHS budgets for hospital building and repair, increase funding for primary medical care, and advocate for an additional £1.5bn increase in public health budgets. Additionally, they support the NHS Reinstatement Bill to protect the NHS from privatization and advocate for increased investment in primary care and public health.""",
        "contexts": [
            "The Green Party pledges to increase NHS spending by £8 billion in the first year, rising to £28 billion by 2030, and keep the NHS fully public."
        ],
        "ground_truth": "Increase NHS spending significantly and ensure the NHS remains fully public."
    },
    {
        "question": "How do the Conservatives plan to reduce immigration?",
        "answer": """The Conservatives plan to reduce immigration by making it clear that if individuals come to the UK illegally, they will not be allowed to stay. They aim to remove the incentive for illegal migration by implementing measures to deter such actions.""",
        "contexts": [
            "The Conservatives plan to introduce a legal cap on migration numbers, remove illegal migrants to Rwanda, and rewrite asylum treaties."
        ],
        "ground_truth": "Legal migration cap, Rwanda deportation policy, rewrite asylum treaties."
    },
    {
        "question": "What is Reform UK's policy on NHS waiting lists?",
        "answer": """Reform UK's policy on NHS waiting lists includes implementing a voucher scheme for private treatment if patients can't see a GP within a certain timeframe, such as 3 days for a GP appointment, 3 weeks for a consultant, and 9 weeks for an operation. They also aim to improve efficiency, cut waste, and unnecessary managers in the NHS to address waiting times.  """,
        "contexts": [
            "Reform UK pledges to achieve zero NHS waiting lists by reforming healthcare delivery, cutting back-office waste, and offering tax breaks to doctors and nurses to tackle staffing shortages."
        ],
        "ground_truth": "Zero NHS waiting lists through healthcare reform, cutting waste, and tax breaks for staff."
    },
    {
        "question": "How will the Liberal Democrats tackle climate change?",
        "answer": """The Liberal Democrats are committed to cutting greenhouse gas emissions to net zero by 2045 at the latest. They will take bold, urgent action to tackle climate change, cut energy bills, and create secure, well-paid new jobs. They plan to make homes warmer and cheaper to heat with a ten-year emergency upgrade program, starting with free insulation and heat pumps for those on low incomes, and ensure that all new homes are zero-carbon. Additionally, they will invest in renewable power, home insulation, and clean energy to drive a strong economic recovery and create clean, secure, well-paid new jobs.""",
        "contexts": [
            "The Liberal Democrats will drive a rooftop solar revolution, invest in clean energy and transport, ban water companies from dumping sewage, and implement an emergency Home Energy Upgrade programme."
        ],
        "ground_truth": "Invest in clean energy, rooftop solar, ban sewage dumping, and upgrade home energy."
    },
    {
        "question": "What energy policy is proposed by Labour in their 2024 manifesto?",
        "answer": """Labour's energy policy proposed in their 2024 manifesto includes deploying more distributed production capacity through their Local Power Plan, partnering with energy companies, local authorities, and co-operatives to install clean power projects such as onshore wind, solar, and hydropower projects. They also plan to incentivize clean energy developers with 
a British Jobs Bonus, invest in onshore wind, solar power, offshore wind, carbon capture and storage, hydrogen, and marine energy. Additionally, Labour aims to establish a new Energy Independence Act to support their energy and climate policies.""",
        "contexts": [
            "Labour proposes creating a publicly-owned clean power company, Great British Energy, to cut bills, boost energy security, and achieve zero-carbon electricity by 2030."
        ],
        "ground_truth": "Create Great British Energy, deliver zero-carbon electricity by 2030."
    },
    {
        "question": "What is the Green Party’s approach to public transport?",
        "answer": """The Green Party's approach to public transport includes championing public transport for every community to thrive, increasing annual public subsidies for rail and bus travel to £10bn, investing an additional £19bn over five years to improve public transport, supporting electrification, investing in new cycleways and footpaths, bringing the railways back into public ownership, giving local authorities control over and funding for improved bus services, ensuring £2.5bn a year is invested in new cycleways and footpaths, and opposing all new road building plans.""",
        "contexts": [
            "The Green Party plans to invest in public transport, including rail, bus, and active travel infrastructure, to reduce car dependency and cut emissions."
        ],
        "ground_truth": "Invest in rail, bus, and active travel infrastructure to cut emissions and car dependency."
    },
    {
        "question": "How do the Conservatives propose to support first-time home buyers?",
        "answer": """The Conservatives propose to support first-time home buyers by ensuring the majority of first-time buyers pay no Stamp Duty at all, making permanent the increase to the threshold at which first-time buyers pay Stamp Duty to £425,000 from £300,000, launching a new and improved Help to Buy scheme to provide first-time buyers with an equity loan of up to 20% towards the cost of a new build home, allowing first-time buyers to get onto the housing ladder with a 5% deposit on interest terms they can afford, and continuing their Mortgage Guarantee Scheme to help households buy a home.""",
        "contexts": [
            "The Conservatives propose abolishing Stamp Duty for homes up to £425,000 for first-time buyers and introducing a new Help to Buy scheme."
        ],
        "ground_truth": "Abolish Stamp Duty for homes up to £425,000 and introduce Help to Buy scheme."
    }
]


In [51]:
# creates hugging face dataset
ragas_dataset = Dataset.from_list(data)

In [52]:
# sets list of metrics to evaluate by
metrics = [
    faithfulness,
    answer_relevancy,
    context_precision,
    context_recall,
    answer_correctness
]

### Results

In [53]:
results = evaluate(ragas_dataset, metrics=metrics)
print(results)

Evaluating: 100%|██████████████████████████████████████████████████████████████████████| 50/50 [00:43<00:00,  1.14it/s]


{'faithfulness': 0.2557, 'answer_relevancy': 0.9596, 'context_precision': 1.0000, 'context_recall': 1.0000, 'answer_correctness': 0.5286}


In [54]:
# creates dataframe of all results for each question
scores_per_sample = pd.DataFrame(results.scores)

# Show per sample scores
scores_per_sample

,faithfulness,answer_relevancy,context_precision,context_recall,answer_correctness
0,0.111111,0.957125,1.0,1.0,0.328750
1,0.333333,0.991475,1.0,1.0,0.729145
2,0.142857,0.876058,1.0,1.0,0.625737
3,0.083333,0.951082,1.0,1.0,0.435451
4,0.750000,0.943391,1.0,1.0,0.386590
5,0.333333,0.957441,1.0,1.0,0.328205
6,0.636364,0.951202,1.0,1.0,0.304141
7,0.000000,0.985169,1.0,1.0,0.208279
8,0.000000,0.997762,1.0,1.0,0.964701
9,0.166667,0.985366,1.0,1.0,0.975199


In [55]:
# Calculate average across all samples
mean_scores = scores_per_sample.mean()
print("\nAverage Scores:")
print(mean_scores)


Average Scores:
faithfulness          0.255700
answer_relevancy      0.959607
context_precision     1.000000
context_recall        1.000000
answer_correctness    0.528620
dtype: float64


## Update made to the k value of retrieved docs.

k number of docs are retrieved for context used by the RAG model. Previously changed this to 10 (from 4) but this may waterdown the context. Changed back to 5

In [56]:
data = [
    {
        "question": "What is Reform UK's policy on immigration?",
        "answer": """Reform UK's policy on immigration includes freezing non-essential immigration, implementing strict limits on immigration to relieve pressure on housing, public services, increase wages, and protect British culture, identity, and values. They also emphasize that essential skills, particularly in healthcare, should be the only exception to these limits. Additionally, Reform UK aims to stop illegal migrants arriving in small boats by detaining and deporting them, as well as potentially returning migrants in small boats to France.""",
        "contexts": [
            "Reform UK proposes to freeze non-essential immigration, stop the small boats, and deport illegal migrants. They also plan to impose a five-year residency requirement for claiming benefits."
        ],
        "ground_truth": "Freeze non-essential immigration, stop small boats, deport illegal migrants, five-year residency rule for benefits."
    },
    {
        "question": "How do the Liberal Democrats plan to improve access to GPs?",
        "answer": """The Liberal Democrats plan to improve access to GPs by giving everyone the right to see a GP within seven days, or within 24 hours if they urgently need to, with 8,000 more GPs to deliver on it. They also aim to introduce a universal 24/7 GP booking system and remove top-down bureaucracy to allow practices to hire the staff they need and invest in training.""",
        "contexts": [
            "Liberal Democrats will give everyone a new right to see a GP within seven days, or 24 hours if urgent, with extra doctors to support this plan."
        ],
        "ground_truth": "Right to see a GP within seven days or 24 hours if urgent, supported by hiring more doctors."
    },
    {
        "question": "What are Labour's five national missions according to the 2024 manifesto?",
        "answer": """Kickstart economic growth to secure the highest sustained growth in the G7 – with good jobs and productivity growth in every part of the country making everyone, not just a few, better off.
2) Make Britain a clean energy superpower to cut bills, create jobs and deliver security with cheaper, zero-carbon electricity by 2030, accelerating to net zero.
3) Take back our streets by halving serious violent crime and raising confidence in the police and criminal justice system to its highest levels.
4) Break down barriers to opportunity by reforming our childcare and education systems, to make sure there is no class ceiling on the ambitions of young people in Britain.
5) Build an NHS fit for the future that is there when people need it; with fewer lives lost to the biggest killers; in a fairer Britain, where everyone lives well for longer.""",
        "contexts": [
            "Labour’s five missions are: kickstart economic growth, make Britain a clean energy superpower, take back our streets, break down barriers to opportunity, and build an NHS fit for the future."
        ],
        "ground_truth": "Kickstart economic growth, clean energy superpower, take back streets, break barriers to opportunity, build a future-ready NHS."
    },
    {
        "question": "What is the Green Party's position on NHS funding?",
        "answer": """The Green Party is committed to fully funding the NHS and keeping it in public hands. They advocate for increasing NHS budgets to meet the demands of an ageing population, reducing waiting lists, boosting the pay of NHS staff, and investing in hospital building and repair. They also prioritize restoring public health budgets and increasing funding for primary medical care.""",
        "contexts": [
            "The Green Party pledges to increase NHS spending by £8 billion in the first year, rising to £28 billion by 2030, and keep the NHS fully public."
        ],
        "ground_truth": "Increase NHS spending significantly and ensure the NHS remains fully public."
    },
    {
        "question": "How do the Conservatives plan to reduce immigration?",
        "answer": """The Conservatives plan to reduce immigration by making it clear that if individuals come to the UK illegally, they will not be allowed to stay. They aim to remove the incentive for illegal migration by implementing measures to deter such actions.""",
        "contexts": [
            "The Conservatives plan to introduce a legal cap on migration numbers, remove illegal migrants to Rwanda, and rewrite asylum treaties."
        ],
        "ground_truth": "Legal migration cap, Rwanda deportation policy, rewrite asylum treaties."
    },
    {
        "question": "What is Reform UK's policy on NHS waiting lists?",
        "answer": """Reform UK's policy on NHS waiting lists is to imagine zero NHS waiting lists by cutting back office waste to spend more money on the frontline, providing tax breaks for doctors and nurses to tackle the staffing crisis.""",
        "contexts": [
            "Reform UK pledges to achieve zero NHS waiting lists by reforming healthcare delivery, cutting back-office waste, and offering tax breaks to doctors and nurses to tackle staffing shortages."
        ],
        "ground_truth": "Zero NHS waiting lists through healthcare reform, cutting waste, and tax breaks for staff."
    },
    {
        "question": "How will the Liberal Democrats tackle climate change?",
        "answer": """The Liberal Democrats are committed to cutting greenhouse gas emissions to net zero by 2045 at the latest. They will take bold, urgent action to tackle climate change, cut energy bills, and create secure, well-paid new jobs. They plan to make homes warmer and cheaper to heat with a ten-year emergency upgrade program, starting with free insulation and heat pumps for those on low incomes, and ensure that all new homes are zero-carbon. Additionally, they will invest in renewable power, home insulation, and clean energy to drive a strong economic recovery and create clean, secure, well-paid new jobs.""",
        "contexts": [
            "The Liberal Democrats will drive a rooftop solar revolution, invest in clean energy and transport, ban water companies from dumping sewage, and implement an emergency Home Energy Upgrade programme."
        ],
        "ground_truth": "Invest in clean energy, rooftop solar, ban sewage dumping, and upgrade home energy."
    },
    {
        "question": "What energy policy is proposed by Labour in their 2024 manifesto?",
        "answer": """Labour proposes a comprehensive energy policy in their 2024 manifesto, including the deployment of more distributed production capacity through their Local Power Plan. They plan to partner with energy companies, local authorities, and co-operatives to install clean power projects such as onshore wind, solar, and hydropower projects. Labour also aims to incentivize clean energy developers with a British Jobs Bonus and invest in ports, hydrogen, and industrial clusters across the country. Additionally, they plan to work with the private sector to increase onshore wind, solar power, and offshore wind, invest in carbon capture and storage, hydrogen, and marine energy, and establish a new Energy Independence Act to support 
their energy and climate policies.""",
        "contexts": [
            "Labour proposes creating a publicly-owned clean power company, Great British Energy, to cut bills, boost energy security, and achieve zero-carbon electricity by 2030."
        ],
        "ground_truth": "Create Great British Energy, deliver zero-carbon electricity by 2030."
    },
    {
        "question": "What is the Green Party’s approach to public transport?",
        "answer": """The Green Party's approach to public transport includes championing public transport for every community to thrive, increasing annual public subsidies for rail and bus travel to £10bn, investing an additional £19bn over five years to improve public transport, supporting electrification, investing in new cycleways and footpaths, bringing the railways back into public ownership, giving local authorities control over and funding for improved bus services, and ensuring £2.5bn a year is invested in new cycleways and footpaths. They also plan to gradually bring train companies back into public ownership and oppose all new road building plans.""",
        "contexts": [
            "The Green Party plans to invest in public transport, including rail, bus, and active travel infrastructure, to reduce car dependency and cut emissions."
        ],
        "ground_truth": "Invest in rail, bus, and active travel infrastructure to cut emissions and car dependency."
    },
    {
        "question": "How do the Conservatives propose to support first-time home buyers?",
        "answer": """The Conservatives propose to support first-time home buyers by ensuring the majority of first-time buyers pay no Stamp Duty at all, making permanent the increase to the threshold at which first-time buyers pay Stamp Duty to £425,000 from £300,000, launching a new and improved Help to Buy scheme to provide first-time buyers with an equity loan of up to 20% towards the cost of a new build home, allowing first-time buyers to get onto the housing ladder with a 5% deposit on interest terms they can afford, and continuing their Mortgage Guarantee Scheme to help households buy a home.""",
        "contexts": [
            "The Conservatives propose abolishing Stamp Duty for homes up to £425,000 for first-time buyers and introducing a new Help to Buy scheme."
        ],
        "ground_truth": "Abolish Stamp Duty for homes up to £425,000 and introduce Help to Buy scheme."
    }
]


In [57]:
# creates hugging face dataset
ragas_dataset = Dataset.from_list(data)

In [58]:
# sets list of metrics to evaluate by
metrics = [
    faithfulness,
    answer_relevancy,
    context_precision,
    context_recall,
    answer_correctness
]

### Results

In [59]:
results = evaluate(ragas_dataset, metrics=metrics)
print(results)

Evaluating: 100%|██████████████████████████████████████████████████████████████████████| 50/50 [00:36<00:00,  1.36it/s]


{'faithfulness': 0.2858, 'answer_relevancy': 0.9631, 'context_precision': 1.0000, 'context_recall': 1.0000, 'answer_correctness': 0.5769}


In [60]:
# creates dataframe of all results for each question
scores_per_sample = pd.DataFrame(results.scores)

# Show per sample scores
scores_per_sample

,faithfulness,answer_relevancy,context_precision,context_recall,answer_correctness
0,0.181818,0.957125,1.0,1.0,0.540166
1,0.285714,0.991482,1.0,1.0,0.680055
2,0.142857,0.872794,1.0,1.0,0.648619
3,0.125000,0.950364,1.0,1.0,0.519280
4,0.750000,0.944758,1.0,1.0,0.199090
5,0.833333,1.000000,1.0,1.0,0.919844
6,0.272727,0.951202,1.0,1.0,0.297891
7,0.000000,0.979691,1.0,1.0,0.211498
8,0.100000,0.997762,1.0,1.0,0.964435
9,0.166667,0.985366,1.0,1.0,0.787699


In [62]:
## Calculate average across all samples
mean_scores = scores_per_sample.mean()
print("\nAverage Scores:")
print(mean_scores)


Average Scores:
faithfulness          0.285812
answer_relevancy      0.963054
context_precision     1.000000
context_recall        1.000000
answer_correctness    0.576858
dtype: float64
